<a href="https://colab.research.google.com/github/koimabrian/GeoAI-NO2-Estimation-Challenge/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming the path to your data and loading it
path = '/content/drive/MyDrive/Colab Notebooks/zindi/GeoAI/'
train_data = pd.read_csv(path + 'Train.csv')
test_data = pd.read_csv(path + 'Test.csv')


In [ ]:
print(train_data.describe())

                LAT           LON  Precipitation           LST           AAI  \
count  86584.000000  86584.000000   86584.000000  46798.000000  73709.000000   
mean      45.421456     10.014272       2.585528    299.268841     -1.274999   
std        0.225409      1.056637       7.617394     10.927167      0.693003   
min       44.924694      8.736497       0.000000    253.200000     -5.196266   
25%       45.249544      9.195325       0.000000    290.920000     -1.749425   
50%       45.478996      9.611738       0.000000    300.760000     -1.349746   
75%       45.601232     10.683357       0.000000    307.940000     -0.878857   
max       45.889734     12.590682     135.396805    327.840000      2.143020   

       CloudFraction     NO2_strat     NO2_total      NO2_trop  \
count   73709.000000  73709.000000  73709.000000  51111.000000   
mean        0.200606      0.000045      0.000156      0.000105   
std         0.259607      0.000012      0.000103      0.000092   
min         0.0

In [ ]:
print(test_data.describe())

               LAT          LON  Precipitation          LST          AAI  \
count  6576.000000  6576.000000    6576.000000  3595.000000  5708.000000   
mean     45.363205    10.489789       2.544018   299.394019    -1.273390   
std       0.260667     1.306469       7.475326    11.034047     0.691949   
min      45.131947     8.842165       0.000000   254.720000    -3.716866   
25%      45.151743     9.146666       0.000000   291.040000    -1.753032   
50%      45.237853    10.398575       0.000000   300.900000    -1.363993   
75%      45.582894    11.642394       0.000000   308.120000    -0.876478   
max      45.836941    12.510362     103.227943   324.140000     2.474583   

       CloudFraction    NO2_strat    NO2_total     NO2_trop  \
count    5708.000000  5708.000000  5708.000000  3998.000000   
mean        0.203512     0.000045     0.000135     0.000081   
std         0.269866     0.000012     0.000084     0.000067   
min         0.000000     0.000013     0.000006    -0.000009   


In [ ]:
# !pip install pysal
import geopandas as gpd
from pysal.lib import weights
from pysal.model import spreg
from sklearn.impute import SimpleImputer
import numpy as np

# Function to process and impute data
def impute_spatial_data(data, required_columns, k_neighbors=10):
    # Ensure all required columns exist in the DataFrame
    for col in required_columns:
        if col not in data.columns:
            data[col] = np.nan

    # Convert the DataFrame to a GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.LON, data.LAT))

    # Drop rows with missing 'LAT' or 'LON' as spatial weights cannot be calculated without coordinates
    gdf = gdf.dropna(subset=['LAT', 'LON'])

    # Create a spatial weights matrix based on k-nearest neighbors
    knn_weights = weights.KNN.from_dataframe(gdf, k=k_neighbors)

    # Fill missing values with initial guesses using SimpleImputer (mean strategy)
    imputer = SimpleImputer(strategy='mean')
    available_features = [col for col in required_columns if col in gdf.columns]
    gdf[available_features] = imputer.fit_transform(gdf[available_features])

    # Spatial Lag Model to handle remaining missing values
    for col in available_features:
        if gdf[col].isnull().sum() > 0:
            # Define the dependent variable (target) and independent variables (features)
            y = gdf[col].values.reshape(-1, 1)
            X = gdf.drop(columns=[col, 'ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'geometry']).values

            # Fit the Spatial Lag Model
            model = spreg.ML_Lag(y, X, w=knn_weights, name_y=col, name_x=gdf.columns.drop([col, 'ID_Zindi', 'Date', 'ID', 'LAT', 'LON', 'geometry']).tolist())

            # Predict missing values
            predicted_values = model.predy.reshape(-1)

            # Replace missing values with predicted ones
            gdf.loc[gdf[col].isnull(), col] = predicted_values[gdf[col].isnull()]

    # Drop the geometry column and return the imputed data
    return gdf.drop(columns=['geometry'])

# Define required columns for train and test data
train_required_columns = ['LST', 'NO2_trop', 'AAI', 'CloudFraction', 'NO2_strat', 'NO2_total', 'TropopausePressure', 'GT_NO2']
test_required_columns = ['LST', 'NO2_trop', 'AAI', 'CloudFraction', 'NO2_strat', 'NO2_total', 'TropopausePressure']  # Excludes 'GT_NO2'

# Process and impute the training data
train_data = impute_spatial_data(train_data, train_required_columns)

# Process and impute the test data
test_data = impute_spatial_data(test_data, test_required_columns)

# # Save the imputed DataFrames
# train_data.to_csv(path + 'Train_Imputed.csv', index=False)
# test_data.to_csv(path + 'Test_Imputed.csv', index=False)

# Check for any remaining missing values in the datasets
print("Training data missing values summary:")
print(train_data.isnull().sum()[train_data.isnull().sum() > 0])

print("Test data missing values summary:")
print(test_data.isnull().sum()[test_data.isnull().sum() > 0])


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.3/347.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.1/243.

/usr/local/lib/python3.10/dist-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 79 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


Training data missing values summary:
Series([], dtype: int64)
Test data missing values summary:
Series([], dtype: int64)


/usr/local/lib/python3.10/dist-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 6 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


In [ ]:
# Convert 'Date' column to datetime and extract temporal features
train_data['Date'] = pd.to_datetime(train_data['Date'])
test_data['Date'] = pd.to_datetime(test_data['Date'])
train_data['Month'] = train_data['Date'].dt.month
test_data['Month'] = test_data['Date'].dt.month
train_data['DayOfWeek'] = train_data['Date'].dt.dayofweek
test_data['DayOfWeek'] = test_data['Date'].dt.dayofweek
train_data['Season'] = train_data['Month'].apply(lambda x: (x%12 + 3)//3)
test_data['Season'] = test_data['Month'].apply(lambda x: (x%12 + 3)//3)

<ipython-input-7-b6fba977caf5>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_data['Date'] = pd.to_datetime(train_data['Date'])
<ipython-input-7-b6fba977caf5>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_data['Date'] = pd.to_datetime(test_data['Date'])


In [ ]:
# # Data preprocessing
# # Handling missing values
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='mean')  # Keep the imputer

# # Impute missing values for all columns with missing data
# for col in train_data.columns:
#     if train_data[col].isnull().any():  # Check if column has missing values
#         train_data[col] = imputer.fit_transform(train_data[[col]])
#         # Only impute on columns that exist in test_data
#         if col in test_data.columns:
#             test_data[col] = imputer.transform(test_data[[col]])

In [ ]:
# Prepare the data for modeling
X_train = train_data.drop(['ID_Zindi', 'Date', 'ID', 'GT_NO2'], axis=1)
y_train = train_data['GT_NO2']
X_test = test_data.drop(['ID_Zindi', 'Date', 'ID'], axis=1)

# Split the dataset for training and validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
# Model development
# Initialize and train XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)

# Initialize and train RandomForest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
preds_xgb = xgb_model.predict(X_val_split)
preds_rf = rf_model.predict(X_val_split)

# Simple Averaging Ensemble
ensemble_preds = 0.5 * preds_xgb + 0.5 * preds_rf

# Evaluate the ensemble model
rmse = np.sqrt(mean_squared_error(y_val_split, ensemble_preds))
print('Ensemble RMSE:', rmse)

# Predict on test data
test_preds_xgb = xgb_model.predict(X_test)
test_preds_rf = rf_model.predict(X_test)
test_ensemble_preds = 0.5 * test_preds_xgb + 0.5 * test_preds_rf

Ensemble RMSE: 6.398530868941271


In [ ]:
# Prepare the submission file with GT_NO2 rounded to two decimal places
submission = pd.DataFrame({
    'ID_Zindi': test_data['ID_Zindi'],
    'GT_NO2': np.round(test_ensemble_preds, 2)
})

# Save the submission file
submission.to_csv(path + 'submission_best_model.csv', index=False)
print(submission)

       ID_Zindi  GT_NO2
0     ID_2MYNQS   30.28
1     ID_P4U5WU   35.95
2     ID_U4KWPK   39.03
3     ID_QGSNTZ   32.42
4     ID_GHSZ6K   33.97
...         ...     ...
6571  ID_GUSXU9   40.16
6572  ID_GMVEG1   36.93
6573  ID_GD6HNP   17.25
6574  ID_J7YW1Y   39.59
6575  ID_I4E04N   31.16

[6576 rows x 2 columns]
